In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer    
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler,Binarizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, StackingClassifier, HistGradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV, KFold, RepeatedKFold, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from catboost import CatBoostClassifier
from IPython.display import clear_output
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.linear_model import LogisticRegressionCV, RidgeCV, LassoCV
from sklearn.preprocessing import PowerTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LassoCV
from sklearn.svm import SVC
from IPython.display import clear_output
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.metrics import roc_auc_score
from skmultilearn.model_selection import IterativeStratification,iterative_stratification
from sklearn.linear_model import LogisticRegression
from optuna.integration import LightGBMPruningCallback
from IPython.display import clear_output
from category_encoders import TargetEncoder, CatBoostEncoder, HelmertEncoder, MEstimateEncoder
# from category_encoders.one_hot import OneHotEncoder
from imblearn.ensemble import BalancedRandomForestClassifier
from scipy.optimize import fmin, differential_evolution
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.special import softmax
import math

In [2]:
data = pd.read_csv('data/train.csv')
original = pd.read_csv('data/original.csv')
test = pd.read_csv('data/test.csv')

In [3]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Machine failure          10000 non-null  int64  
 9   TWF                      10000 non-null  int64  
 10  HDF                      10000 non-null  int64  
 11  PWF                      10000 non-null  int64  
 12  OSF                      10000 non-null  int64  
 13  RNF                      10000 non-null  int64  
dtypes: float64(3), int64(9)

In [4]:
new_data = pd.concat([
                      data.drop(columns=['id']), 
                      original.drop(columns=['UDI']),                   
                     ], 
                     ignore_index = True)
new_data.shape

(146429, 13)

In [5]:
# new_data = data
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146429 entries, 0 to 146428
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Product ID               146429 non-null  object 
 1   Type                     146429 non-null  object 
 2   Air temperature [K]      146429 non-null  float64
 3   Process temperature [K]  146429 non-null  float64
 4   Rotational speed [rpm]   146429 non-null  int64  
 5   Torque [Nm]              146429 non-null  float64
 6   Tool wear [min]          146429 non-null  int64  
 7   Machine failure          146429 non-null  int64  
 8   TWF                      146429 non-null  int64  
 9   HDF                      146429 non-null  int64  
 10  PWF                      146429 non-null  int64  
 11  OSF                      146429 non-null  int64  
 12  RNF                      146429 non-null  int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 14.5+ MB


In [6]:
new_data = new_data.drop_duplicates()

In [7]:
new_data[new_data['Air temperature [K]'] == 0.00 ]

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF


In [8]:
# new_data['Product ID'] = new_data['Product ID'].str.slice(start=1).astype(float)
# new_data['Product ID'].dtypes

In [9]:
new_data['Machine failure'].value_counts(normalize=True)

0    0.983014
1    0.016986
Name: Machine failure, dtype: float64

In [10]:
class_weights = {0: 0.016986, 1: 0.983014}

In [11]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [12]:
# new_data = reduce_mem_usage(new_data)

In [13]:
import re
def clean_feature_names(data):
    cleaned_columns = []
    for column in data.columns:
        cleaned_column = re.sub('[^A-Za-z0-9_]+', '', column)
        cleaned_columns.append(cleaned_column)
    data.columns = cleaned_columns
    return data

In [14]:
len(new_data.columns)

13

In [15]:
new_data = clean_feature_names(new_data)
# Fit and Transform new_data
# Base_Prob = 0.0
# # Calculate the mean of Machine.failure for each Product.ID
# train_prob = new_data.groupby('ProductID')['Machinefailure'].mean().reset_index()
# train_prob.columns = ['ProductID', 'prob']
# # Merge the calculated means into the original dataframe
# new_data = pd.merge(new_data, train_prob, on='ProductID', how='inner')
# # Replace prob values less than or equal to 0.15 with Base_Prob
# new_data['prob'] = np.where(new_data['prob'] <= 0.15, Base_Prob, new_data['prob'])

In [16]:
new_data.select_dtypes(exclude='O').columns

Index(['AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm',
       'TorqueNm', 'Toolwearmin', 'Machinefailure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')

In [17]:
np.max(new_data['Toolwearmin'])

253

In [18]:
X = new_data.drop(columns=['Machinefailure'])
y = new_data['Machinefailure']

In [19]:
imb_ratio = float(np.sum(y == 0)) / np.sum(y==1)
imb_ratio

57.87155591572123

In [20]:
X.columns

Index(['ProductID', 'Type', 'AirtemperatureK', 'ProcesstemperatureK',
       'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'TWF', 'HDF', 'PWF',
       'OSF', 'RNF'],
      dtype='object')

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 20px; color: #1192AA; text-align:left">Preprocessing</h2>

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 20px; color: #1192AA; text-align:left">Light GBM</h2>

In [21]:
# lgb_best_params = 
#                   {'n_estimators': 608, 
#                    'learning_rate': 0.008786927731607481, 
#                    'max_depth': 10, 
#                    'num_leaves': 963, 
#                    'subsample': 0.5404369780011518, 
#                    'colsample_bytree': 0.507815726575903,
#                    'is_unbalance' : True, 
#                    'metric' : 'auc'
#                   }

lgb_best_params = {
                     'num_leaves': 64, 
                     'min_child_samples': 175, 
                     'max_depth': 18, 
                     'learning_rate': 0.013397950211504788, 
                     'colsample_bytree': 0.82, 
                     'n_estimators': 400, 
                     'reg_alpha': 4.798968170715771, 
                     'reg_lambda': 0.8001065724847404, 
                     'min_split_gain': 0.0227379006101743, 
                     'min_child_weight': 4.189021119554275, 
                     'subsample_freq': 2, 
                     'subsample': 0.7, 
                     'max_bin': 140,
                     'categorical_feature' : [0,1],
                   }

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 20px; color: #1192AA; text-align:left">XGBoost</h2>

In [22]:
xgb_best_params = { 
                    'n_estimators': 600,
                    'max_depth': 6,
                    'eval_metric' : 'auc',
                    'lambda': 0.01, 
                    'alpha': 0.01, 
                    'colsample_bytree': 0.7, 
                    'subsample': 0.5, 
                    'learning_rate': 0.008, 
                    'min_child_weight': 1,
                  }

xgb_best_params1 = { 'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
                    'n_estimators': 800,
                    'max_depth':7,
                    'scale_pos_weight' : imb_ratio,
                    'eval_metric' : 'auc',
                    'lambda': 0.0005, 
                    'alpha': 0.028372243586991808, 
                    'colsample_bytree': 0.3944892085183995, 
                    'subsample': 0.840671541957931, 
                    'learning_rate': 0.01, 
                    'min_child_weight': 1,
                    'predictor' : 'gpu_predictor'
                  }

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 20px; color: #1192AA; text-align:left">CatBoost</h2>

In [23]:
# cb_best_params= {'n_estimators': 361, 
#                  'learning_rate': 0.03732997222102321, 
#                  'max_depth': 8, 
#                  'subsample': 0.6959508889112623, 
#                  'colsample_bylevel': 0.5032559564347243,
#                  'silent' : True,
#                  'auto_class_weights': 'Balanced',
#                  'eval_metric': 'AUC',
#                  'random_strength': 43,
#                 }

cb_best_params = {   'iterations': 1200,
                     'bootstrap_type': 'Bayesian',
                     'depth': 1,
                     'learning_rate': 0.08911870285993972,
                     'random_strength': 82,
                     'border_count': 210,
                     'l2_leaf_reg': 8.928917322292204,
                     'boosting_type': 'Ordered',
                     'bagging_temperature': 7.167555149152137,
                     'silent' : True,
                 }

In [24]:
hgbc_best_params = {'learning_rate': 0.1862615382501618, 
                   'max_iter': 934, 
                   'max_depth': 5, 
                   'l2_regularization': 0.31942345884883905,
                   'class_weight' : 'balanced'
                  }

In [25]:
numeric_cols = [   'AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm',
                   'TorqueNm', 'Toolwearmin', 'TWF', 'HDF', 'PWF', 'OSF', 'Temp_Diff',
                   'Failure_Summary', 'AirtemperatureKSquared', 'AirtemperatureKCubed',
                   'AirtemperatureKLog', 'ProcesstemperatureKSquared',
                   'ProcesstemperatureKCubed', 'ProcesstemperatureKLog',
                   'RotationalspeedrpmSquared', 'RotationalspeedrpmCubed',
                   'RotationalspeedrpmLog', 'TorqueNmSquared', 'TorqueNmCubed',
                   'TorqueNmLog', 'ToolwearminSquared', 'ToolwearminCubed',
                   'ToolwearminLog'
               ]

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin
class CustomFeats(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, df, y=None):
        return self

    def transform(self, df, y=None):
        df = df.copy()
        df['Temp_Diff'] = df['AirtemperatureK'] - df['ProcesstemperatureK']
#         df['RotationalSpeed_TorqueRatio'] = df['Rotationalspeedrpm'] / df['TorqueNm']
#         df['Power'] = df['TorqueNm'] * df['Rotationalspeedrpm']
#         df['EfficiencyIndex'] = df['Power'] / df['Toolwearmin']

#         # Calculate temperature difference
#         df['TemperatureDifference'] = df['ProcesstemperatureK'] - df['AirtemperatureK']
#         # Calculate temperature variability
#         df['TemperatureVariability'] = df[['AirtemperatureK', 'ProcesstemperatureK']].std(axis=1)
#         # Calculate temperature ratio
#         df['TemperatureRatio'] = df['ProcesstemperatureK'] / df['AirtemperatureK']
#         #Calculcating Average Temperature
#         df['AverageTemperature'] = (df['ProcesstemperatureK'] + df['AirtemperatureK'])/2
        
#         # Calculate tool wear rate
#         max_tool_wear = df['Toolwearmin'].max()
#         df['ToolWearRate'] = df['Toolwearmin'] / max_tool_wear
#         # Calculate temperature change rate
#         df['TemperatureChangeRate'] = df['TemperatureDifference'] / (np.where(df['Toolwearmin']==0, 2,df['Toolwearmin']))
#         df['Torque_ToolwearRatio'] = df['TorqueNm'] / (np.where(df['Toolwearmin']==0, 2,df['Toolwearmin']))

        df['Failure_Summary'] = df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1)
        
        df = df.drop(['RNF'], axis =1)
        
        features_list = ['AirtemperatureK', 'ProcesstemperatureK', 'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin']
        for feat in features_list:
            df[f'{feat}Squared'] = df[feat] ** 2
            df[f'{feat}Cubed'] = df[feat] ** 3
            df[f'{feat}Log'] = df[feat].apply(lambda x: math.log(x) if x > 0 else 0)
         
        return df


In [27]:
def replace_outliers(data):
    for col in data[numeric_cols]:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower = Q1 - 1.5*IQR
        upper = Q3 + 1.5*IQR
        
        if col not in ['Rotationalspeedrpm', 'Power']:
            data[col] = np.clip(data[col], lower, upper)
        else: 
            data[col] = np.clip(data[col], lower-200, upper+200)

    return data

In [28]:
# Now, you can define your pipelines without 'custom_feats'

num_pipe = Pipeline([
#     ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
    ('scaler', RobustScaler()),
])

cat_pipe = Pipeline([
#     ('encoder', OneHotEncoder(handle_unknown="ignore")),
    ('encoder', CatBoostEncoder(cols = ['ProductID', 'Type'])),
])

# Update your preprocessor to use this pipeline for numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
#         ('num', num_pipe, numeric_cols),
        ('cat', cat_pipe, ['ProductID', 'Type']),
    ], remainder='passthrough')

preprocessing_steps = [('Preprocessor', preprocessor)]

In [29]:
# custom_feats = CustomFeats()
# X = custom_feats.fit_transform(X)
# X.select_dtypes(exclude='O').columns

In [30]:
cat_features =  X.select_dtypes('O').columns
for col in cat_features:
    X[col] = X[col].astype('category')

In [31]:
model_pipes = []
# Base Layer 1
lgbm_pipe = Pipeline(steps=preprocessing_steps + [('lgbm', lgb.LGBMClassifier(**lgb_best_params))])
model_pipes.append(lgbm_pipe)

# Base Layer 2
xgb_pipe = Pipeline(steps=preprocessing_steps + [('xgb', XGBClassifier(**xgb_best_params))])
model_pipes.append(xgb_pipe)

# # Base Layer 3
cb_pipe = Pipeline(steps=preprocessing_steps + [('cb', CatBoostClassifier(**cb_best_params))])
model_pipes.append(cb_pipe)

# # # Base Layer 4
# xgb_pipe1 = Pipeline(steps=preprocessing_steps + [('xgb2', XGBClassifier(**xgb_best_params1))])
# model_pipes.append(xgb_pipe1)


# # # Base Layer 5
# brf_pipe = Pipeline(steps=preprocessing_steps + [('brf', BalancedRandomForestClassifier(  n_estimators=1000, 
#                                                                                            max_depth=None, 
#                                                                                            min_samples_split=2, 
#                                                                                            min_samples_leaf=1, 
#                                                                                            class_weight='balanced'))])
# model_pipes.append(brf_pipe)

# # Base Layer 6
# hgbc_pipe = Pipeline(steps=preprocessing_steps + [('hgbc', HistGradientBoostingClassifier(**hgbc_best_params))])
# model_pipes.append(hgbc_pipe)


<h2 style = "font-family: Georgia;font-weight: bold; font-size: 20px; color: #1192AA; text-align:left">Blending to find the optimum CV</h2>

In [32]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# This will suppress all warnings
warnings.filterwarnings("ignore")

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 25px; color: #1192AA; text-align:left">Blending</h2>

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 15px; color: #1192AA; text-align:left">1. Averaging</h2>

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 15px; color: #1192AA; text-align:left">2. Rank Averaging </h2>

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 15px; color: #1192AA; text-align:left">3. Weighted Averaging( calculated using f-min) </h2>

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 15px; color: #1192AA; text-align:left">3. Weighted Averaging( calculated using Differential Evolution) </h2>

In [33]:
%%time
kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

# Initialize dictionary to hold AUC for each method
auc_dict = {
    "Weighted_Averaging": [],
    "Simple_Averaging": [],
    "Rank_Averaging": [],
    "Differential_Evolution" : [] ,
    'Voting' : [],
}

# Loop over folds
for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f'Fold: {fold}')
   
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
    
    # First apply the custom features to your data
    custom_feats = CustomFeats()
    X_train_fold = custom_feats.fit_transform(X_train_fold)
    X_val_fold = custom_feats.transform(X_val_fold)

    # replacing outliers
#     X_train_fold = replace_outliers(X_train_fold)
#     X_val_fold = replace_outliers(X_val_fold)
        
    pred_cols = []
    optimized_base_models = []

    # Create a new DataFrame for this fold
    blended_df_fold = pd.DataFrame(index=range(len(X_val_fold)))

    # Train each model on the current fold
    for model_pipe in model_pipes:
        model = list(model_pipe.named_steps.keys())[1]
        print('Training model=', model, end=' ')
        model_pipe.fit(X_train_fold, y_train_fold)
        
        # Getting the name of the model        
        model_name = model+'_valid_pred_fold_'+str(fold)
        pred_cols.append(model_name)

        # Store predictions for this fold
        blended_df_fold[model_name] = model_pipe.predict_proba(X_val_fold)[:,1]
        print(f'auc={roc_auc_score(y_val_fold, blended_df_fold[model_name])}')
    
    # Optimize weights on this fold using differential evolution
    def objective_de(weights: np.ndarray):
        weights = weights / np.sum(weights)  # Ensure weights are normalized
        weighted_avg_preds = np.average(blended_df_fold[pred_cols].values, axis=1, weights=weights)
        return -roc_auc_score(y_val_fold, weighted_avg_preds)
    
    # define bounds for differential_evolution
    bounds = [(0,1)] * len(model_pipes)
    result = differential_evolution(objective_de, bounds)
    optimal_weights_de = result.x
    optimal_weights_de = optimal_weights_de / np.sum(optimal_weights_de)  # Normalize weights
    
    # Add the differential evolution predictions to the DataFrame
    blended_df_fold['diff_evolution_fold_'+str(fold)] = np.average(blended_df_fold[pred_cols].values, 
                                                                   axis=1, 
                                                                   weights=optimal_weights_de)

    auc_diff_evolution = roc_auc_score(y_val_fold, blended_df_fold['diff_evolution_fold_'+str(fold)])
    auc_dict["Differential_Evolution"].append(auc_diff_evolution)
    
    # Optimize weights on this fold using F-Min
    def objective(weights: np.ndarray):
        weights = weights / np.sum(weights)  # Ensure weights are normalized
        weighted_avg_preds = np.average(blended_df_fold[pred_cols].values, axis=1, weights=weights)
        return -roc_auc_score(y_val_fold, weighted_avg_preds)

    weights_init = np.array([1.0 / len(model_pipes)] * len(model_pipes))  # Initialize weights
    optimal_weights = fmin(objective, weights_init, disp=True)  # Find weights that minimize the negative AUC
    optimal_weights = optimal_weights / np.sum(optimal_weights)  # Normalize weights

    # Add the weighted average predictions to the DataFrame
    blended_df_fold['weighted_avg_fold_'+str(fold)] = np.average(blended_df_fold[pred_cols].values, 
                                                                 axis=1, 
                                                                 weights=optimal_weights)

    auc_weighted_avg = roc_auc_score(y_val_fold, blended_df_fold['weighted_avg_fold_'+str(fold)])
    auc_dict["Weighted_Averaging"].append(auc_weighted_avg)

    # Simple averaging
    blended_df_fold['simple_avg_fold_'+str(fold)] = blended_df_fold[pred_cols].mean(axis=1)
    auc_simple_avg = roc_auc_score(y_val_fold, blended_df_fold['simple_avg_fold_'+str(fold)])
    auc_dict["Simple_Averaging"].append(auc_simple_avg)

    # Rank averaging
    blended_df_fold['rank_avg_fold_'+str(fold)] = blended_df_fold[pred_cols].rank().mean(axis=1)
    auc_rank_avg = roc_auc_score(y_val_fold, blended_df_fold['rank_avg_fold_'+str(fold)])
    auc_dict["Rank_Averaging"].append(auc_rank_avg)   

# Calculate mean AUC for each method
for method in auc_dict:
    auc_dict[method] = np.mean(auc_dict[method])

# Convert dictionary to DataFrame
auc_df = pd.DataFrame.from_dict(auc_dict, orient='index', columns=['Mean_AUC'])

clear_output()
auc_df

CPU times: total: 2h 25min 25s
Wall time: 45min 46s


,Mean_AUC
Weighted_Averaging,0.981711
Simple_Averaging,0.980495
Rank_Averaging,0.977979
Differential_Evolution,0.981493
Voting,NaN


In [34]:
X_train_fold.columns

Index(['ProductID', 'Type', 'AirtemperatureK', 'ProcesstemperatureK',
       'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'TWF', 'HDF', 'PWF',
       'OSF', 'Temp_Diff', 'Failure_Summary', 'AirtemperatureKSquared',
       'AirtemperatureKCubed', 'AirtemperatureKLog',
       'ProcesstemperatureKSquared', 'ProcesstemperatureKCubed',
       'ProcesstemperatureKLog', 'RotationalspeedrpmSquared',
       'RotationalspeedrpmCubed', 'RotationalspeedrpmLog', 'TorqueNmSquared',
       'TorqueNmCubed', 'TorqueNmLog', 'ToolwearminSquared',
       'ToolwearminCubed', 'ToolwearminLog'],
      dtype='object')

In [35]:
blended_df_fold.head()

,lgbm_valid_pred_fold_14,xgb_valid_pred_fold_14,cb_valid_pred_fold_14,diff_evolution_fold_14,weighted_avg_fold_14,simple_avg_fold_14,rank_avg_fold_14
0,0.000652,0.004962,0.000965,0.004540,0.003426,0.002193,5918.666667
1,0.000624,0.005025,0.001006,0.004598,0.003479,0.002219,8752.000000
2,0.000658,0.004962,0.000939,0.004538,0.003417,0.002186,4300.666667
3,0.000934,0.004957,0.001111,0.004555,0.003481,0.002334,9913.666667
4,0.001077,0.005134,0.001082,0.004717,0.003584,0.002431,14010.000000


In [36]:
optimal_weights

array([0.02482228, 0.61753653, 0.35764119])

In [37]:
optimal_weights_de

array([0.03514297, 0.89709319, 0.06776384])

In [38]:
X.columns

Index(['ProductID', 'Type', 'AirtemperatureK', 'ProcesstemperatureK',
       'Rotationalspeedrpm', 'TorqueNm', 'Toolwearmin', 'TWF', 'HDF', 'PWF',
       'OSF', 'RNF'],
      dtype='object')

In [39]:
%%time

# First apply the custom features to your data
custom_feats = CustomFeats()
X = custom_feats.fit_transform(X)

# replacing outliers
# X = replace_outliers(X)  

from sklearn.calibration import CalibratedClassifierCV
# Train each model on the full training set
optimized_base_models = []
for model_pipe in model_pipes:
    model = list(model_pipe.named_steps.keys())[1]
    print(f'Training: {model} on entire data')  
    model_pipe.fit(X, y)
    
    # Getting the name of the model        
    model_name = model+'_trained_on_entire_data'
    
    # Calibrate the model using cross validation
    calibrator = CalibratedClassifierCV(model_pipe, cv=5, method='sigmoid')
    calibrator.fit(X, y)
    
    optimized_base_models.append((model_name, calibrator))
    
print(f'\nTraining completed for all {len(optimized_base_models)} models')

Training: lgbm on entire data
Training: xgb on entire data
Training: cb on entire data

Training completed for all 3 models
CPU times: total: 53min 12s
Wall time: 6min 22s


In [40]:
len(optimized_base_models)

3

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 20px; color: #1192AA; text-align:left">Predicting probabilities on Test Set:</h2>

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 15px; color: #1192AA; text-align:left">Using Blending:</h2>

In [41]:
# test['Product ID'] = test['Product ID'].str.slice(start=1).astype(float)
# test['Type'] = type_enc.transform(test['Type'])
test = clean_feature_names(test)

# # Encoding Product ID
# test = pd.merge(test, train_prob, on='ProductID', how='left')
# test['prob'].fillna(Base_Prob, inplace=True)
# test['prob'] = np.where(test['prob'] <= 0.15, Base_Prob, test['prob'])

In [42]:
for col in cat_features:
    test[col] = test[col].astype('category')

In [43]:
%%time
test = custom_feats.transform(test)
# test = replace_outliers(test)

# Initialize a DataFrame to store test set predictions
test_preds = pd.DataFrame(data={'id' : test.id})

# Calculate test set predictions for each model
for model_name, calibrator in optimized_base_models:
    print(f'Predicting for model {model_name}')
    test_preds[model_name] = calibrator.predict_proba(test)[:,1]

pred_cols = [col for col in test_preds.columns if col != 'id']

# Apply the best method to combine the predictions
if auc_df.idxmax().values[0] == "Weighted_Averaging":
    # Calculate the weighted average of the test set predictions
    # Use the same weights as the ones that gave the best AUC on the validation set
    test_preds['Machine failure'] = np.average(test_preds[pred_cols].values, axis=1, weights=optimal_weights)
elif auc_df.idxmax().values[0] == "Simple_Averaging":
    # Calculate the simple average of the test set predictions
    test_preds['Machine failure'] = test_preds[pred_cols].mean(axis=1)
elif auc_df.idxmax().values[0] == "Rank_Averaging":
#     Calculate the rank average of the test set predictions
    test_preds['Machine failure'] = test_preds[pred_cols].rank().mean(axis=1)    
elif auc_df.idxmax().values[0] == "Differential_Evolution":
#     Calculate the rank average of the test set predictions
    test_preds['Machine failure'] = np.average(test_preds[pred_cols].values, axis=1, weights=optimal_weights_de)

Predicting for model lgbm_trained_on_entire_data
Predicting for model xgb_trained_on_entire_data
Predicting for model cb_trained_on_entire_data
CPU times: total: 42.1 s
Wall time: 2.76 s


In [44]:
test_preds[pred_cols]

,lgbm_trained_on_entire_data,xgb_trained_on_entire_data,cb_trained_on_entire_data
0,0.003288,0.002687,0.002959
1,0.003307,0.002686,0.002962
2,0.003275,0.002683,0.002959
3,0.003318,0.002699,0.002948
4,0.003300,0.002687,0.002953
...,...,...,...
90949,0.003294,0.002687,0.002961
90950,0.003281,0.002764,0.002978
90951,0.003323,0.002693,0.003003
90952,0.003278,0.003391,0.003168


In [45]:
test_preds[['id', 'Machine failure']].to_csv('blend_robust_w8_avg_calibrated_fmin_6mdls_catboostEnc_cb_lgbm_tuned.csv', index=False)

<h2 style = "font-family: Georgia;font-weight: bold; font-size: 15px; color: #1192AA; text-align:left">Using Stacking:</h2>